In [1]:
from code_data_science import data_table as dt, data_grid as moderne_data_grid
import json
import math 
import re
import gradio_client
from collections import Counter


def clean_text(text):
    #clean input text as much as possible, to allow parsing a maximum of recommendations
    text = text.replace("\n", " ")
    pattern = r'```.*?```'
    text = re.sub(pattern, '', text, flags=re.DOTALL)
    pattern = r'```[^`]*\"'
    text = re.sub(pattern, '"', text, flags=re.DOTALL)
    text = re.sub(r"`([^`]*)`", lambda m: "`" + m.group(1).replace('"', '') + "`", text)
    return text

def parse_recommendations(str):
    try:
        return json.loads(clean_text(str))
    except json.JSONDecodeError as e:
        return math.nan


# get data
df = dt.read_csv("../samples/recommendations.csv")
print(len(df))
columns = ["repositoryPath", "repositoryBranch", "Recommendations"]
df = df[columns]
df["Recommendations"] = df["Recommendations"].apply(parse_recommendations)
df.dropna(subset=["Recommendations"], inplace=True)

40


In [9]:
client = gradio_client.Client("http://127.0.0.1:7863")

recipes = []
for item in df["Recommendations"]:
    for recommendation in item:
        recipes_results = json.loads(client.predict(recommendation, 25, 0, fn_index=1).replace("'", "\""))
        recipes.extend([recipe["recipe_id"] for recipe in recipes_results[:min(5, len(recipes_results))]])

final_df = dt.DataFrame(Counter(recipes).items(), columns=["recipe_id", "count"])

# reorder by count
final_df = final_df.sort_values(by="count", ascending=False)

moderne_data_grid.display(final_df)

Loaded as API: http://127.0.0.1:7863/ ✔
